In [1]:
# https://arxiv.org/pdf/quant-ph/0104030.pdf
# ^^^ Need to be able to prepare arbitrary state!
import numpy as np
import cirq
from functools import reduce

In [2]:
def Get_state_as_str(n_qubits, qubit_state_int):
    """
    converts qubit state int into binary form.

    Args:
        n_qubits (int): Number of qubits
        qubit_state_int (int): qubit state as int (NOT BINARY!)
    Returns:
        string of qubit state in binary!

    state = |000> + |001> + |010> + |011> + |100> + |101 > + |110 > + |111>
    state  = |0> +   |1> +   |2> +   |3> +   |4> +   |5 > +   |6 > +   |7>

    n_qubits = 3
    state = 5
    Get_state_as_str(n_qubits, state)
    >> '101'

    """
    bin_str_len = '{' + "0:0{}b".format(n_qubits) + '}'
    return bin_str_len.format(qubit_state_int)

In [3]:
class My_U_Gate(cirq.SingleQubitGate):
    """
    Description

    Args:
        theta (float): angle to rotate by in radians.
        number_control_qubits (int): number of control qubits
    """

    def __init__(self, theta):
        self.theta = theta
    def _unitary_(self):
        Unitary_Matrix = np.array([
                    [np.cos(self.theta), np.sin(self.theta)],
                    [np.sin(self.theta), -1* np.cos(self.theta)]
                ])
        return Unitary_Matrix
    def num_qubits(self):
        return 1

    def _circuit_diagram_info_(self,args):
        # return cirq.CircuitDiagramInfo(
        #     wire_symbols=tuple([*['@' for _ in range(self.num_control_qubits-1)],' U = {} rad '.format(self.theta.__round__(4))]),exponent=1)
        return ' U = {} rad '.format(self.theta.__round__(4))

In [4]:
theta = np.pi

U_single_qubit = My_U_Gate(theta)
op = U_single_qubit.on(cirq.LineQubit(1))
print(cirq.Circuit(op))

print('####')

op = U_single_qubit.controlled(num_controls=3, control_values=[0, 0, 1]).on(*[cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)], cirq.LineQubit(4))
print(cirq.Circuit(op))

1: ─── U = 3.1416 rad ───
####
1: ───(0)────────────────
      │
2: ───(0)────────────────
      │
3: ───@──────────────────
      │
4: ─── U = 3.1416 rad ───


$a_{000}|000\rangle+a_{001}|001\rangle+a_{010}|010\rangle+$
$a_{011}|011\rangle+a_{100}|100\rangle+a_{101}|101\rangle+a_{110}|110\rangle+a_{111}|111\rangle$

$$\alpha_{j, i_{1} i_{2} \cdots i_{j-1}}=\arctan \sqrt{\frac{\sum_{i_{j+1} \cdots i_{n}}\left|a_{i_{1} i_{2} \cdots i_{j-1} 1 i_{j+1} \cdots i_{n}}\right|^{2}}{\sum_{i_{j+1} \cdots i_{n}}\left|a_{i_{1} i_{2} \cdots i_{j-1} 0 i_{j+1} \cdots i_{n}}\right|^{2}}}$$


example

$\alpha_{3, 01}$ means target qubit is 3 and is controlled by 01 state!

In [ ]:
# def Get_control_parameters(num_qubits, coefficient_list):
    
#     if len(coefficient_list)!= 2**num_qubits:
#         raise ValueError('incorrect number of coefficients')
    
#     state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
#     alpha_j_dict={}
#     for target_qubit in range(num_qubits):

#         if target_qubit==0:
#             top=0
#             bottom=0

#             CONTROL_state=None            
#             for index, qubit_term in enumerate(state_list):                 
#                 if qubit_term[target_qubit]=='0':
#                     top+=Coefficient_list[index]**2
#                 elif qubit_term[target_qubit]=='1':
#                     bottom+=Coefficient_list[index]**2
#                 else:
#                     print('MISTAKE')


#             alpha_j_dict[target_qubit]= [{'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))}]

#         else:
#             CONTROL_state_list = [Get_state_as_str(target_qubit, i) for i in range(2**target_qubit)]

#             term_list=[]
#             for CONTROL_state in CONTROL_state_list:
#                 top=0
#                 bottom=0

#                 for index, qubit_term in enumerate(state_list):                 
#                     if qubit_term[target_qubit]=='0':
#                         top+=Coefficient_list[index]**2
#                     elif qubit_term[target_qubit]=='1':
#                         bottom+=Coefficient_list[index]**2
#                     else:
#                         print('MISTAKE')
#                 term_list.append({'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))})
#             alpha_j_dict[target_qubit]= term_list
#     return alpha_j_dict

# Get_control_parameters(2, [np.sqrt(0), np.sqrt(0), np.sqrt(0.1), np.sqrt(0.9)]) 

In [ ]:
# class State_Prep_Circuit(cirq.Gate):
#     """
#     Function to build cirq Circuit that will make an arbitrary state!

#     e.g.:
#     {
#          0: [{'control_state': None, 'angle': 0.7853981633974483}],
#          1: [{'control_state': '0', 'angle': 0.7853981633974483},
#           {'control_state': '1', 'angle': 0.7853981633974483}]
#       }

# gives :

# 0: ── U = 0.51 rad ──(0)─────────────@──────────────(0)────────────(0)──────────────@────────────────@────────────────
#                      │               │              │              │                │                │
# 1: ────────────────── U = 0.91 rad ── U = 0.93 rad ─(0)────────────@────────────────(0)──────────────@────────────────
#                                                     │              │                │                │
# 2: ───────────────────────────────────────────────── U = 0.30 rad ─ U = 0.59 rad ─── U = 0.72 rad ─── U = 0.71 rad ───

#     Args:
#         circuit_param_dict (dict): A Dictionary of Tuples (qubit, control_val(int)) value is angle

#     Returns
#         A cirq circuit object to be used by cirq.Circuit.from_ops to generate arbitrary state

#     """
#     def __init__(self, circuit_param_dict):

#         self.circuit_param_dict = circuit_param_dict

#     def _decompose_(self, qubits):

#         for qubit in self.circuit_param_dict:
            
#             for term in self.circuit_param_dict[qubit]:
#                 if term['control_state']:
#                     control_values=[int(bit) for bit in term['control_state']]
#                     num_controls = len(control_values)
#                     theta = term['angle']
#                     qubit_list = cirq.LineQubit.range(0,1+num_controls)
                    
#                     yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)
#                 else:
#                     theta = term['angle']
#                     yield My_U_Gate(theta).on(cirq.LineQubit(qubit))
                    
#     def _circuit_diagram_info_(self, args):

#         max_qubit = max(self.circuit_param_dict.keys())
#         string_list = []
#         for i in range(max_qubit):
#             string_list.append('state prep circuit')
#         return string_list

#     def num_qubits(self):
#         return max(self.circuit_param_dict.keys())

In [ ]:
# alpha_j = Get_control_parameters(2, [np.sqrt(0), np.sqrt(0), np.sqrt(0.1), np.sqrt(0.9)]) 

# state_circ_obj = State_Prep_Circuit(alpha_j)

# circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
# print(circuit)

In [111]:
# def Get_control_parameters(num_qubits, Coefficient_list):
    
#     if len(Coefficient_list)!= 2**num_qubits:
#         raise ValueError('incorrect number of coefficients')
    
#     state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
#     alpha_j_dict={}
#     for target_qubit in range(num_qubits-1):

#         number_controls = target_qubit

#         if number_controls >0:
#             CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
#         else:
#             CONTROL_state_list = ['']


#         term_list=[]
#         for control_state in CONTROL_state_list:
#             top_term_str = control_state + '1'
#             bottom_term_str = control_state + '0'

#             top=0
#             bottom=0
#             for index, state_str in enumerate(state_list): 
#                 if state_str[:target_qubit+1]==top_term_str:
#                     top+=Coefficient_list[index]**2

#                 if state_str[:target_qubit+1]==bottom_term_str:
#                     bottom+=Coefficient_list[index]**2
#                 else:
#                     continue
                    
#             if (bottom ==0) and (top==0):
#                 angle=0
#             else:
#                 angle = np.arctan(top/bottom)
                
#             term_list.append({'control_state':control_state,'angle': angle})
#         alpha_j_dict[target_qubit]= term_list

#     ##final rotation ##

#     term_list=[]
#     for index, control_state_str in enumerate([Get_state_as_str((num_qubits-1), i) for i in range(2**(num_qubits-1))]):

#         top_term_str = control_state_str + '1'
#         bottom_term_str = control_state_str + '0'

#         index_top = state_list.index(top_term_str)
#         index_bottom = state_list.index(bottom_term_str)

#         top= Coefficient_list[index_top]
#         bottom=  Coefficient_list[index_bottom]

#         if (bottom ==0) and (top==0):
#             angle=0
#         else:
#             angle = np.arctan(top/bottom)

#         term_list.append({'control_state':control_state_str,'angle': angle})

#     alpha_j_dict[num_qubits-1]= term_list
    
#     return alpha_j_dict

# Get_control_parameters(2, [np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.3), np.sqrt(0.4)]) 

In [112]:
# def Get_control_parameters(num_qubits, Coefficient_list):
    
#     if len(Coefficient_list)!= 2**num_qubits:
#         raise ValueError('incorrect number of coefficients')
    
#     state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
#     alpha_j_dict={}
#     for target_qubit in range(num_qubits-1):

#         number_controls = target_qubit

#         if number_controls >0:
#             CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
#         else:
#             CONTROL_state_list = ['']


#         term_list=[]
#         for control_state in CONTROL_state_list:
#             top_term_str = control_state + '1'
#             bottom_term_str = control_state + '0'

#             top=0
#             bottom=0
#             for index, state_str in enumerate(state_list): 
#                 if state_str[:target_qubit+1]==top_term_str:
#                     top+=Coefficient_list[index]**2

#                 if state_str[:target_qubit+1]==bottom_term_str:
#                     bottom+=Coefficient_list[index]**2
#                 else:
#                     continue
                    
#             if (bottom ==0) and (top==0):
#                 angle=0
#             else:
#                 try:                                 #<--- new try and except!
#                     angle = np.arctan(top/bottom)
#                 except:
#                     angle=0
#             term_list.append({'control_state':control_state,'angle': angle})
#         alpha_j_dict[target_qubit]= term_list

#     ##final rotation ##

#     term_list=[]
#     for index, control_state_str in enumerate([Get_state_as_str((num_qubits-1), i) for i in range(2**(num_qubits-1))]):

#         top_term_str = control_state_str + '1'
#         bottom_term_str = control_state_str + '0'

#         index_top = state_list.index(top_term_str)
#         index_bottom = state_list.index(bottom_term_str)

#         top= Coefficient_list[index_top]
#         bottom=  Coefficient_list[index_bottom]

#         if (bottom ==0) and (top==0):
#             angle=0
#         else:
#             try:
#                 angle = np.arctan(top/bottom)
#             except:
#                 angle=0

#         term_list.append({'control_state':control_state_str,'angle': angle})

#     alpha_j_dict[num_qubits-1]= term_list
    
#     return alpha_j_dict

# Get_control_parameters(2, [np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.3), np.sqrt(0.4)]) 

In [201]:
# def Get_control_parameters(num_qubits, Coefficient_list):
    
#     if len(Coefficient_list)!= 2**num_qubits:
#         raise ValueError('incorrect number of coefficients')
    
#     state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
#     alpha_j_dict={}
#     for target_qubit in range(num_qubits-1):

#         number_controls = target_qubit

#         if number_controls >0:
#             CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
#         else:
#             CONTROL_state_list = ['']


#         term_list=[]
#         for control_state in CONTROL_state_list:
#             top_term_str = control_state + '1'
#             bottom_term_str = control_state + '0'

#             top=0
#             bottom=0
#             for index, state_str in enumerate(state_list): 
#                 if state_str[:target_qubit+1]==top_term_str:
#                     top+=Coefficient_list[index]**2

#                 if state_str[:target_qubit+1]==bottom_term_str:
#                     bottom+=Coefficient_list[index]**2
#                 else:
#                     continue
            
#             if (bottom ==0) and (top==0):
#                 angle=0
#             else:
#                 angle = np.arctan(np.sqrt(top/bottom))
#             term_list.append({'control_state':control_state,'angle': angle})
#         alpha_j_dict[target_qubit]= term_list

#     ##final rotation ##

#     term_list=[]
#     for index, state_str in enumerate([Get_state_as_str((num_qubits-1), i) for i in range(2**(num_qubits-1))]):
#         control_state_str=state_str

#         top_term_str = control_state_str + '1'
#         bottom_term_str = control_state_str + '0'

#         index_top = state_list.index(top_term_str)
#         index_bottom = state_list.index(bottom_term_str)

#         top= Coefficient_list[index_top]
#         bottom=  Coefficient_list[index_bottom]

#         if (bottom ==0) and (top==0):
#             angle=0
#         else:
#             angle = np.arctan(top/bottom)

#         term_list.append({'control_state':control_state_str,'angle': angle})

#     alpha_j_dict[num_qubits-1]= term_list

#     return alpha_j_dict

# Get_control_parameters(2, [np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.3), np.sqrt(0.4)]) 

{0: [{'control_state': '', 'angle': 0.9911565864311923}],
 1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

In [207]:
def Get_control_parameters(num_qubits, Coefficient_list):
    
    if len(Coefficient_list)!= 2**num_qubits:
        raise ValueError('incorrect number of coefficients')
    
    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
    alpha_j_dict={}
    for target_qubit in range(num_qubits-1):

        number_controls = target_qubit

        if number_controls >0:
            CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
        else:
            CONTROL_state_list = ['']


        term_list=[]
        for control_state in CONTROL_state_list:
            top_term_str = control_state + '1'
            bottom_term_str = control_state + '0'

            top=0
            bottom=0
            for index, state_str in enumerate(state_list): 
                if state_str[:target_qubit+1]==top_term_str:
                    top+=Coefficient_list[index]**2

                if state_str[:target_qubit+1]==bottom_term_str:
                    bottom+=Coefficient_list[index]**2
                else:
                    continue
            
            if (bottom ==0) and (top==0):
                angle=0
            else:
                try:
                    angle = np.arctan(np.sqrt(top/bottom))
                except:
                    raise ValueError('undetermined angle! NEED TO CHECK PROBLEM')
                    
            term_list.append({'control_state':control_state,'angle': angle})
        alpha_j_dict[target_qubit]= term_list

    ##final rotation ##

    term_list=[]
    for index, state_str in enumerate([Get_state_as_str((num_qubits-1), i) for i in range(2**(num_qubits-1))]):
        control_state_str=state_str

        top_term_str = control_state_str + '1'
        bottom_term_str = control_state_str + '0'

        index_top = state_list.index(top_term_str)
        index_bottom = state_list.index(bottom_term_str)

        top= Coefficient_list[index_top]
        bottom=  Coefficient_list[index_bottom]

        if (bottom ==0) and (top==0):
            angle=0
        else:
            try:
                angle = np.arctan(top/bottom)
            except:
                raise ValueError('undetermined angle! NEED TO CHECK PROBLEM')

        term_list.append({'control_state':control_state_str,'angle': angle})

    alpha_j_dict[num_qubits-1]= term_list

    return alpha_j_dict

Get_control_parameters(2, [np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.3), np.sqrt(0.4)]) 

{0: [{'control_state': '', 'angle': 0.9911565864311923}],
 1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

In [223]:
def Get_control_parameters(num_qubits, Coefficient_list):
    
    if len(Coefficient_list)!= 2**num_qubits:
        raise ValueError('incorrect number of coefficients')
    
    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
    alpha_j_dict={}
    for target_qubit in range(num_qubits-1):

        number_controls = target_qubit

        if number_controls >0:
            CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
        else:
            CONTROL_state_list = ['']


        term_list=[]
        for control_state in CONTROL_state_list:
            top_term_str = control_state + '1'
            bottom_term_str = control_state + '0'

            top=0
            bottom=0
            for index, state_str in enumerate(state_list): 
                if state_str[:target_qubit+1]==top_term_str:
                    top+=Coefficient_list[index]**2

                if state_str[:target_qubit+1]==bottom_term_str:
                    bottom+=Coefficient_list[index]**2
                else:
                    continue
            
            if (bottom ==0) and (top==0):
                angle=0
            else:
                try:
                    angle = np.arctan(np.sqrt(top/bottom))
                except:
                    angle = 0
                    
            term_list.append({'control_state':control_state,'angle': angle})
        alpha_j_dict[target_qubit]= term_list

    ##final rotation ##

    term_list=[]
    for index, state_str in enumerate([Get_state_as_str((num_qubits-1), i) for i in range(2**(num_qubits-1))]):
        control_state_str=state_str

        top_term_str = control_state_str + '1'
        bottom_term_str = control_state_str + '0'

        index_top = state_list.index(top_term_str)
        index_bottom = state_list.index(bottom_term_str)

        top= Coefficient_list[index_top]
        bottom=  Coefficient_list[index_bottom]

        if (bottom ==0) and (top==0):
            angle=0
        else:
            try:
                angle = np.arctan(top/bottom)
            except:
                angle=0

        term_list.append({'control_state':control_state_str,'angle': angle})

    alpha_j_dict[num_qubits-1]= term_list

    return alpha_j_dict

Get_control_parameters(2, [np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.3), np.sqrt(0.4)]) 

{0: [{'control_state': '', 'angle': 0.9911565864311923}],
 1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

In [224]:
class State_Prep_Circuit(cirq.Gate):
    """
    Function to build cirq Circuit that will make an arbitrary state!

    e.g.:
    {
         0: [{'control_state': None, 'angle': 0.7853981633974483}],
         1: [{'control_state': '0', 'angle': 0.7853981633974483},
          {'control_state': '1', 'angle': 0.7853981633974483}]
      }

gives :

0: ── U = 0.51 rad ──(0)─────────────@──────────────(0)────────────(0)──────────────@────────────────@────────────────
                     │               │              │              │                │                │
1: ────────────────── U = 0.91 rad ── U = 0.93 rad ─(0)────────────@────────────────(0)──────────────@────────────────
                                                    │              │                │                │
2: ───────────────────────────────────────────────── U = 0.30 rad ─ U = 0.59 rad ─── U = 0.72 rad ─── U = 0.71 rad ───

    Args:
        circuit_param_dict (dict): A Dictionary of Tuples (qubit, control_val(int)) value is angle

    Returns
        A cirq circuit object to be used by cirq.Circuit.from_ops to generate arbitrary state

    """
    def __init__(self, circuit_param_dict):

        self.circuit_param_dict = circuit_param_dict

    def _decompose_(self, qubits):

        for qubit in self.circuit_param_dict:
            
            for term in self.circuit_param_dict[qubit]:
                if term['control_state']:
                    control_values=[int(bit) for bit in term['control_state']]
                    num_controls = len(control_values)
                    theta = term['angle']
                    
#                     if theta==0:
#                         yield cirq.I.on(cirq.LineQubit(qubit))
#                     else:
#                         U_single_qubit = My_U_Gate(theta)
#                         qubit_list = cirq.LineQubit.range(0,1+num_controls)
#                         yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)
                    U_single_qubit = My_U_Gate(theta)
                    qubit_list = cirq.LineQubit.range(0,1+num_controls)
                    yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)
                else:
                    theta = term['angle']
#                     if theta==0:
#                         yield cirq.I.on(cirq.LineQubit(qubit))
#                     else:
#                         yield My_U_Gate(theta).on(cirq.LineQubit(qubit))
                    theta = term['angle']
                    yield My_U_Gate(theta).on(cirq.LineQubit(qubit))
                    
    def _circuit_diagram_info_(self, args):

        max_qubit = max(self.circuit_param_dict.keys())
        string_list = []
        for i in range(max_qubit):
            string_list.append('state prep circuit')
        return string_list

    def num_qubits(self):
        return max(self.circuit_param_dict.keys())

In [208]:
w =My_U_Gate(0)
w._unitary_()

array([[ 1.,  0.],
       [ 0., -1.]])

In [209]:
alpha_j

{0: [{'control_state': '', 'angle': 1.5707963267948966}],
 1: [{'control_state': '0', 'angle': 0},
  {'control_state': '1', 'angle': 0.4636476090008061}],
 2: [{'control_state': '00', 'angle': 0},
  {'control_state': '01', 'angle': 0},
  {'control_state': '10', 'angle': 0.0},
  {'control_state': '11', 'angle': 0.0}]}

In [225]:
coef = [np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.3), np.sqrt(0.4)]
n_qubits=2

alpha_j = Get_control_parameters(n_qubits, coef) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
print(circuit)


from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('')
print('state')
print(Get_wavefunction(circuit))
print('')
print(coef)

0: ─── U = 0.9912 rad ───(0)────────────────@──────────────────
                         │                  │
1: ────────────────────── U = 0.9553 rad ─── U = 0.8571 rad ───

state
[[0.316+0.j]
 [0.447+0.j]
 [0.548+0.j]
 [0.632+0.j]]

[0.31622776601683794, 0.4472135954999579, 0.5477225575051661, 0.6324555320336759]


In [226]:
coef = [0, 0, 0, 0,  np.sqrt(0.8), 0, np.sqrt(0.2), 0]
n_qubits=3

alpha_j = Get_control_parameters(n_qubits, coef) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
print(circuit)


from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('')
print('state')
print(Get_wavefunction(circuit))
print('')
print(coef)

0: ─── U = 1.5708 rad ───(0)───────────@──────────────────(0)───────────(0)───────────@───────────────@───────────────
                         │             │                  │             │             │               │
1: ────────────────────── U = 0 rad ─── U = 0.4636 rad ───(0)───────────@─────────────(0)─────────────@───────────────
                                                          │             │             │               │
2: ─────────────────────────────────────────────────────── U = 0 rad ─── U = 0 rad ─── U = 0.0 rad ─── U = 0.0 rad ───

state
[[0.   +0.j]
 [0.   +0.j]
 [0.   +0.j]
 [0.   +0.j]
 [0.894+0.j]
 [0.   +0.j]
 [0.447+0.j]
 [0.   +0.j]]

[0, 0, 0, 0, 0.8944271909999159, 0, 0.4472135954999579, 0]


/home/lex/anaconda3/envs/QuantumUpdated/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars


In [228]:
coef = [0., 0., 0., 0.11028503978328148, 0., 0., 0., 0., 0., 0., 0., 0., 0.9939, 0., 0., 0.]
# coef = [np.sqrt(1/16)for _ in range(2**4)]
n_qubits=4

alpha_j = Get_control_parameters(n_qubits, coef) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
print(circuit)


from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('')
print('state')
print(Get_wavefunction(circuit))
print('')
print(coef)

0: ─── U = 1.4603 rad ───(0)─────────────@─────────────(0)───────────(0)───────────@─────────────@───────────────(0)───────────(0)───────────(0)───────────(0)───────────@─────────────@─────────────@───────────────@─────────────
                         │               │             │             │             │             │               │             │             │             │             │             │             │               │
1: ────────────────────── U = 0.0 rad ─── U = 0 rad ───(0)───────────@─────────────(0)───────────@───────────────(0)───────────(0)───────────@─────────────@─────────────(0)───────────(0)───────────@───────────────@─────────────
                                                       │             │             │             │               │             │             │             │             │             │             │               │
2: ──────────────────────────────────────────────────── U = 0 rad ─── U = 0 rad ─── U = 0 rad ─── U = 0.0 rad ───(

In [214]:
coef = [np.sqrt(0.125) for _ in range(8)]
n_qubits=3

alpha_j = Get_control_parameters(n_qubits, coef) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
print(circuit)


from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('')
print('state')
dd= Get_wavefunction(circuit)
print('')
print(coef)

0: ─── U = 0.7854 rad ───(0)────────────────@──────────────────(0)────────────────(0)────────────────@──────────────────@──────────────────
                         │                  │                  │                  │                  │                  │
1: ────────────────────── U = 0.7854 rad ─── U = 0.7854 rad ───(0)────────────────@──────────────────(0)────────────────@──────────────────
                                                               │                  │                  │                  │
2: ──────────────────────────────────────────────────────────── U = 0.7854 rad ─── U = 0.7854 rad ─── U = 0.7854 rad ─── U = 0.7854 rad ───

state

[0.3535533905932738, 0.3535533905932738, 0.3535533905932738, 0.3535533905932738, 0.3535533905932738, 0.3535533905932738, 0.3535533905932738, 0.3535533905932738]


In [ ]:
# # MEASURE
# qubits_to_measure = (cirq.LineQubit(q_No) for q_No in range(n_qubits))
# circuit.append(cirq.measure(*qubits_to_measure))
# print(circuit)

# # simulate
# simulator = cirq.Simulator()
# results = simulator.run(circuit, repetitions=10000)
# print(results.histogram(key='0,1')) # Need key to match number of qubits!!!

In [123]:
num_qubits=2
state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
Coefficient_list= [np.sqrt(0.1),np.sqrt(0.2),np.sqrt(0.3),np.sqrt(0.4)]# [1/2,1/2,1/2,1/2]#[1/np.sqrt(8) for _ in range(2 ** num_qubits)]#[1/2,1/2,1/2,1/2] #np.random.rand(2 ** num_qubits)

alpha_j_dict={}
for target_qubit in range(num_qubits-1):
    
    number_controls = target_qubit
    
    if number_controls >0:
        CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
    else:
        CONTROL_state_list = ['']
    
    
    term_list=[]
    for control_state in CONTROL_state_list:
        top_term_str = control_state + '1'
        bottom_term_str = control_state + '0'
        
        top=0
        bottom=0
        for index, state_str in enumerate(state_list): 
            if state_str[:target_qubit+1]==top_term_str:
                top+=Coefficient_list[index]**2
                
            if state_str[:target_qubit+1]==bottom_term_str:
                bottom+=Coefficient_list[index]**2
            else:
                continue
        term_list.append({'control_state':control_state,'angle': np.arctan(np.sqrt(top/bottom))})
    alpha_j_dict[target_qubit]= term_list

##final rotation ##

term_list=[]
for index, state_str in enumerate([Get_state_as_str((num_qubits-1), i) for i in range(2**(num_qubits-1))]):
    control_state_str=state_str
    
    top_term_str = control_state_str + '1'
    bottom_term_str = control_state_str + '0'
    
    index_top = state_list.index(top_term_str)
    index_bottom = state_list.index(bottom_term_str)
    
    top= Coefficient_list[index_top]
    bottom=  Coefficient_list[index_bottom]
    
    if (bottom ==0) and (top==0):
        angle=0
    else:
        angle = np.arctan(top/bottom)
        
    term_list.append({'control_state':control_state_str,'angle': angle})
    
alpha_j_dict[num_qubits-1]= term_list
    
alpha_j_dict

{0: [{'control_state': '', 'angle': 0.9911565864311923}],
 1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

# Manual

$$\alpha_{1}=\arctan \sqrt{\frac{\left|a_{10}\right|^{2}+\left|a_{11}\right|^{2}}{\left|a_{00}\right|^{2}+\left|a_{01}\right|^{2}}}$$

$$U_{\alpha_{2,0}}=\left[\begin{array}{cc}\frac{a_{00}}{\sqrt{\left|a_{00}\right|^{2}+\left|a_{01}\right|^{2}}} & \frac{a_{01}}{\sqrt{\left|a_{00}\right|^{2}+\left|a_{01}\right|^{2}}} \\ \frac{a_{01}^{*}}{\sqrt{\left|a_{00}\right|^{2}+\left|a_{01}\right|^{2}}} & -\frac{a_{00}^{*}}{\sqrt{\left|a_{00}\right|^{2}+\left|a_{01}\right|^{2}}}\end{array}\right]$$

$$U_{\alpha_{2,1}}=\left[\begin{array}{cc}\frac{a_{10}}{\sqrt{\left|a_{10}\right|^{2}+\left|a_{11}\right|^{2}}} & \frac{a_{11}}{\sqrt{\left|a_{10}\right|^{2}+\left|a_{11}\right|^{2}}} \\ \frac{a_{11}^{*}}{\sqrt{\left|a_{10}\right|^{2}+\left|a_{11}\right|^{2}}} & -\frac{a_{10}^{*}}{\sqrt{\left|a_{10}\right|^{2}+\left|a_{11}\right|^{2}}}\end{array}\right]$$

In [124]:
state_list = ['00', '01', '10', '11']
Coefficient_list= [np.sqrt(0.1),np.sqrt(0.2),np.sqrt(0.3),np.sqrt(0.4)]

In [125]:
a_1_coef= (Coefficient_list[2]**2+Coefficient_list[3]**2)/(Coefficient_list[0]**2+Coefficient_list[1]**2)
a_1 = np.arctan(np.sqrt(a_1_coef))
U_1=My_U_Gate(a_1)._unitary_()
U_1

array([[ 0.54772256,  0.83666003],
       [ 0.83666003, -0.54772256]])

In [126]:
U_2_0 = np.array([
    [Coefficient_list[0]/np.sqrt(Coefficient_list[0]**2 + Coefficient_list[1]**2 ), Coefficient_list[1]/np.sqrt(Coefficient_list[0]**2 + Coefficient_list[1]**2 )],
    [Coefficient_list[1]/np.sqrt(Coefficient_list[0]**2 + Coefficient_list[1]**2 ),(-1)*(Coefficient_list[0]/np.sqrt(Coefficient_list[0]**2 + Coefficient_list[1]**2))]
        ])
U_2_0

array([[ 0.57735027,  0.81649658],
       [ 0.81649658, -0.57735027]])

In [127]:
U_2_1 = np.array([
    [Coefficient_list[2]/np.sqrt(Coefficient_list[2]**2 + Coefficient_list[3]**2 ), Coefficient_list[3]/np.sqrt(Coefficient_list[2]**2 + Coefficient_list[3]**2 )],
    [Coefficient_list[3]/np.sqrt(Coefficient_list[2]**2 + Coefficient_list[3]**2 ),(-1)*(Coefficient_list[2]/np.sqrt(Coefficient_list[2]**2 + Coefficient_list[3]**2))]
        ])
U_2_1

array([[ 0.65465367,  0.75592895],
       [ 0.75592895, -0.65465367]])

In [128]:
a_1_coef= (Coefficient_list[2]**2+Coefficient_list[3]**2)/(Coefficient_list[0]**2+Coefficient_list[1]**2)
a_1 = np.arctan(np.sqrt(a_1_coef))
a_1

0.9911565864311923

In [129]:
a_2_0_coef = (Coefficient_list[1])/(Coefficient_list[0])
a_2_0 = np.arctan(a_2_0_coef)
a_2_0

0.9553166181245092

In [130]:
a_2_1_coef = (Coefficient_list[3])/(Coefficient_list[2])
a_2_1 = np.arctan(a_2_1_coef)
a_2_1

0.8570719478501311

In [131]:
U_20=My_U_Gate(a_2_0)._unitary_()
U_20 == U_2_0

array([[ True,  True],
       [ True,  True]])

In [132]:
U_21=My_U_Gate(a_2_1)._unitary_()
U_21 == U_2_1

array([[ True,  True],
       [ True,  True]])

In [133]:
test = {0: [{'control_state': '', 'angle': a_1}],
 1: [{'control_state': '0', 'angle': a_2_0},
  {'control_state': '1', 'angle': a_2_1}]}
test

{0: [{'control_state': '', 'angle': 0.9911565864311923}],
 1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

In [134]:
Get_control_parameters(2, Coefficient_list) 

{0: [{'control_state': '', 'angle': 0.9911565864311923}],
 1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

In [138]:
state_circ_obj = State_Prep_Circuit(test)
circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
print(circuit)
from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('')
print('state')
print(Get_wavefunction(circuit))
print('')
print(Coefficient_list)

0: ─── U = 0.9912 rad ───(0)────────────────@──────────────────
                         │                  │
1: ────────────────────── U = 0.9553 rad ─── U = 0.8571 rad ───

state
[[0.316+0.j]
 [0.447+0.j]
 [0.548+0.j]
 [0.632+0.j]]

[0.31622776601683794, 0.4472135954999579, 0.5477225575051661, 0.6324555320336759]


In [ ]:
Get_control_parameters(2, Coefficient_list)

In [ ]:
class State_Prep_Circuit(cirq.Gate):
    """
    Function to build cirq Circuit that will make an arbitrary state!

    e.g.:
    {
         0: [{'control_state': None, 'angle': 0.7853981633974483}],
         1: [{'control_state': '0', 'angle': 0.7853981633974483},
          {'control_state': '1', 'angle': 0.7853981633974483}]
      }

gives :

0: ── U = 0.51 rad ──(0)─────────────@──────────────(0)────────────(0)──────────────@────────────────@────────────────
                     │               │              │              │                │                │
1: ────────────────── U = 0.91 rad ── U = 0.93 rad ─(0)────────────@────────────────(0)──────────────@────────────────
                                                    │              │                │                │
2: ───────────────────────────────────────────────── U = 0.30 rad ─ U = 0.59 rad ─── U = 0.72 rad ─── U = 0.71 rad ───

    Args:
        circuit_param_dict (dict): A Dictionary of Tuples (qubit, control_val(int)) value is angle

    Returns
        A cirq circuit object to be used by cirq.Circuit.from_ops to generate arbitrary state

    """
    def __init__(self, circuit_param_dict):

        self.circuit_param_dict = circuit_param_dict

    def _decompose_(self, qubits):

        for qubit in self.circuit_param_dict:
            
            for term in self.circuit_param_dict[qubit]:
                if term['control_state']:
                    control_values=[int(bit) for bit in term['control_state']]
                    num_controls = len(control_values)
                    theta = term['angle']
                    
#                     if theta==0:
#                         pass
#                     else:
#                         U_single_qubit = My_U_Gate(theta)
#                         qubit_list = cirq.LineQubit.range(0,1+num_controls)
#                         yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)
                    U_single_qubit = My_U_Gate(theta)
                    qubit_list = cirq.LineQubit.range(0,1+num_controls)
                    yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)
                else:
#                     theta = term['angle']
#                     if theta==0:
#                         pass
#                     else:
#                         yield My_U_Gate(theta).on(cirq.LineQubit(qubit))
                    theta = term['angle']
                    yield My_U_Gate(theta).on(cirq.LineQubit(qubit))
                    
    def _circuit_diagram_info_(self, args):

        max_qubit = max(self.circuit_param_dict.keys())
        string_list = []
        for i in range(max_qubit):
            string_list.append('state prep circuit')
        return string_list

    def num_qubits(self):
        return max(self.circuit_param_dict.keys())

In [ ]:
# num_qubits=2
# state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
# Coefficient_list= [np.sqrt(0.1),np.sqrt(0.2),np.sqrt(0.3),np.sqrt(0.4)]# [1/2,1/2,1/2,1/2]#[1/np.sqrt(8) for _ in range(2 ** num_qubits)]#[1/2,1/2,1/2,1/2] #np.random.rand(2 ** num_qubits)

# alpha_j_dict={}
# for target_qubit in range(num_qubits):
    
#     number_controls = target_qubit
    
#     if number_controls >0:
#         CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
#     else:
#         CONTROL_state_list = ['']
    
    
#     term_list=[]
#     for control_state in CONTROL_state_list:
#         top_term_str = control_state + '1'
#         bottom_term_str = control_state + '0'
        
#         top=0
#         bottom=0
#         for index, state_str in enumerate(state_list): 
#             if state_str[:target_qubit+1]==top_term_str:
#                 top+=Coefficient_list[index]**2
                
#             if state_str[:target_qubit+1]==bottom_term_str:
#                 bottom+=Coefficient_list[index]**2
#             else:
#                 continue
#         term_list.append({'control_state':control_state,'angle': np.arctan(np.sqrt(top/bottom))})
#     alpha_j_dict[target_qubit]= term_list
    
# alpha_j_dict

# state_circ_obj = State_Prep_Circuit(alpha_j_dict)
# circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
# print(circuit)
# from quchem.Simulating_Quantum_Circuit import Get_wavefunction
# print('')
# print('state')
# dd= Get_wavefunction(circuit)
# print(Coefficient_list)

In [ ]:
for qubit in xx:
    for term in xx[qubit]:
        if term['control_state']:
            control_values=[int(bit) for bit in term['control_state']]
            print(control_values)
            theta = term['angle']
            print(theta)

In [ ]:
class State_Prep_Circuit(cirq.Gate):
    """
    Function to build cirq Circuit that will make an arbitrary state!

    e.g.:
    {
         0: [{'control_state': None, 'angle': 0.7853981633974483}],
         1: [{'control_state': '0', 'angle': 0.7853981633974483},
          {'control_state': '1', 'angle': 0.7853981633974483}]
      }

gives :

0: ── U = 0.51 rad ──(0)─────────────@──────────────(0)────────────(0)──────────────@────────────────@────────────────
                     │               │              │              │                │                │
1: ────────────────── U = 0.91 rad ── U = 0.93 rad ─(0)────────────@────────────────(0)──────────────@────────────────
                                                    │              │                │                │
2: ───────────────────────────────────────────────── U = 0.30 rad ─ U = 0.59 rad ─── U = 0.72 rad ─── U = 0.71 rad ───

    Args:
        circuit_param_dict (dict): A Dictionary of Tuples (qubit, control_val(int)) value is angle

    Returns
        A cirq circuit object to be used by cirq.Circuit.from_ops to generate arbitrary state

    """
    def __init__(self, circuit_param_dict):

        self.circuit_param_dict = circuit_param_dict

    def _decompose_(self, qubits):

        for qubit in self.circuit_param_dict:
            
            for term in self.circuit_param_dict[qubit]:
                if term['control_state']:
                    control_values=[int(bit) for bit in term['control_state']]
                    num_controls = len(control_values)
                    theta = term['angle']
                    qubit_list = cirq.LineQubit.range(0,1+num_controls)
                    
                    yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)
                else:
                    theta = term['angle']
                    yield My_U_Gate(theta).on(cirq.LineQubit(qubit))
                    
    def _circuit_diagram_info_(self, args):

        max_qubit = max(self.circuit_param_dict.keys())
        string_list = []
        for i in range(max_qubit):
            string_list.append('state prep circuit')
        return string_list

    def num_qubits(self):
        return max(self.circuit_param_dict.keys())

In [ ]:
alpha_j = Get_control_parameters(2, [np.sqrt(0), np.sqrt(0), np.sqrt(0.1), np.sqrt(0.9)]) 

state_circ_obj = State_Prep_Circuit(alpha_j)

circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))
print(circuit)

In [ ]:
    e.g.:
   {
        (1, 0): 0.5092156980522868,
        (2, 0): 0.9097461710606383,
        (2, 1): 0.9338960671361634,
        (3, 0): 0.3007458481278772,
        (3, 1): 0.5945638342986989,
        (3, 2): 0.7174996992546281,
        (3, 3): 0.7105908988639925
    }


In [ ]:
['cat','dog','bat','cow'][1:]

In [ ]:
num_qubits  = 3
for target_qubit in range(num_qubits):
    
    if target_qubit==0:
        CONTROL_state_list=['']
    else:
        CONTROL_state_list = [Get_state_as_str(target_qubit, i) for i in range(2**target_qubit)]
    print(CONTROL_state_list)
    

In [ ]:
num_qubits=3
state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
Coefficient_list= [1/np.sqrt(8) for _ in range(2 ** num_qubits)]#[1/2,1/2,1/2,1/2] #np.random.rand(2 ** num_qubits)

alpha_j_dict={}
for target_qubit in range(num_qubits-1):
    
    number_controls = target_qubit
    
    if number_controls >0:
        CONTROL_state_list = [Get_state_as_str(number_controls, i) for i in range(2**number_controls)]
    else:
        CONTROL_state_list = ['']
    
    
    term_list=[]
    for control_state in CONTROL_state_list:
        top_term_str = control_state + '1'
        bottom_term_str = control_state + '0'
        
        top=0
        bottom=0
        for index, state_str in enumerate(state_list): 
            if state_str[:target_qubit+1]==top_term_str:
                top+=Coefficient_list[index]**2
                
            if state_str[:target_qubit+1]==bottom_term_str:
                bottom+=Coefficient_list[index]**2
            else:
                continue
        term_list.append({'control_state':control_state,'angle': np.arctan(np.sqrt(top/bottom))})
    alpha_j_dict[target_qubit]= term_list

##final rotation ##

term_list=[]
for index, state_str in enumerate(state_list):
    control_state_str=state_str[:-1]
    
    top_term_str = control_state_str + '1'
    bottom_term_str = control_state_str + '0'
    
    index_top = state_list.index(top_term_str)
    index_bottom = state_list.index(bottom_term_str)
    
    top= Coefficient_list[index_top]
    bottom=  Coefficient_list[index_bottom]
    term_list.append({'control_state':control_state_str,'angle': np.arctan(np.sqrt(top/bottom))})
alpha_j_dict[num_qubits-1]= term_list


#     for index, state_str in enumerate(state_list): 
#         if state_str[:target_qubit+1]==top_term_str:
#             top+=Coefficient_list[index]**2

#         if state_str[:target_qubit+1]==bottom_term_str:
#             bottom+=Coefficient_list[index]**2
#         else:
#             continue
#     term_list.append({'control_state':control_state_str,'angle': np.arctan(np.sqrt(top/bottom))})
# alpha_j_dict[target_qubit]= term_list
    
    
    
alpha_j_dict

In [ ]:
f = ['12','13','14']
f.index('13')

In [ ]:
num_qubits=3
state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
Coefficient_list= [1/np.sqrt(8) for _ in range(2 ** num_qubits)]
state_list

In [ ]:
Coefficient_list[2]**2+Coefficient_list[3]**2

In [ ]:
Coefficient_list[0]**2+Coefficient_list[1]**2

In [ ]:
for target_qubit in range(num_qubits):
    if target_qubit==0:
        CONTROL_state_list=[]
    else:
        CONTROL_state_list = [Get_state_as_str(target_qubit, i) for i in range(2**target_qubit)]
    
    for control_state in CONTROL_state_list:
        
    

In [ ]:
num_qubits=2
state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
Coefficient_list= [1/2,1/2,1/2,1/2] #np.random.rand(2 ** num_qubits)

alpha_j_dict={}
for target_qubit in range(num_qubits):
    
    if target_qubit==0:
        top=0
        bottom=0
        
        CONTROL_state=None            
        for index, qubit_term in enumerate(state_list):                 
            if qubit_term[target_qubit]=='0':
                top+=Coefficient_list[index]**2
            elif qubit_term[target_qubit]=='1':
                bottom+=Coefficient_list[index]**2
            else:
                print('MISTAKE')
        

        alpha_j_dict[target_qubit]= [{'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))}]
        
    else:
        CONTROL_state_list = [Get_state_as_str(target_qubit, i) for i in range(2**target_qubit)]
        
        term_list=[]
        for CONTROL_state in CONTROL_state_list:
            top=0
            bottom=0
            
            for index, qubit_term in enumerate(state_list):                 
                if qubit_term[target_qubit]=='0':
                    top+=Coefficient_list[index]**2
                elif qubit_term[target_qubit]=='1':
                    bottom+=Coefficient_list[index]**2
                else:
                    print('MISTAKE')
            term_list.append({'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))})
        alpha_j_dict[target_qubit]= term_list

alpha_j_dict

In [ ]:
num_qubits=3
state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
Coefficient_list= [1/np.sqrt(8) for _ in range(2 ** num_qubits)]

alpha_j_dict={}
for target_qubit in range(num_qubits):
    
    if target_qubit==0:
        top=0
        bottom=0
        
        CONTROL_state=None            
        for index, qubit_term in enumerate(state_list):                 
            if qubit_term[target_qubit]=='0':
                top+=Coefficient_list[index]**2
            elif qubit_term[target_qubit]=='1':
                bottom+=Coefficient_list[index]**2
            else:
                print('MISTAKE')
        

        alpha_j_dict[target_qubit]= [{'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))}]
        
    else:
        CONTROL_state_list = [Get_state_as_str(target_qubit, i) for i in range(2**target_qubit)]
        
        term_list=[]
        for CONTROL_state in CONTROL_state_list:
            top=0
            bottom=0
            
            for index, qubit_term in enumerate(state_list):                 
                if qubit_term[target_qubit]=='0':
                    top+=Coefficient_list[index]**2
                elif qubit_term[target_qubit]=='1':
                    bottom+=Coefficient_list[index]**2
                else:
                    print('MISTAKE')
            term_list.append({'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))})
        alpha_j_dict[target_qubit]= term_list

alpha_j_dict

In [ ]:
def Get_control_parameters(num_qubits, coefficient_list):
    
    if len(coefficient_list)!= 2**num_qubits:
        raise ValueError('incorrect number of coefficients')
    
    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    
    alpha_j_dict={}
    for target_qubit in range(num_qubits):

        if target_qubit==0:
            top=0
            bottom=0

            CONTROL_state=None            
            for index, qubit_term in enumerate(state_list):                 
                if qubit_term[target_qubit]=='0':
                    top+=Coefficient_list[index]**2
                elif qubit_term[target_qubit]=='1':
                    bottom+=Coefficient_list[index]**2
                else:
                    print('MISTAKE')


            alpha_j_dict[target_qubit]= [{'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))}]

        else:
            CONTROL_state_list = [Get_state_as_str(target_qubit, i) for i in range(2**target_qubit)]

            term_list=[]
            for CONTROL_state in CONTROL_state_list:
                top=0
                bottom=0

                for index, qubit_term in enumerate(state_list):                 
                    if qubit_term[target_qubit]=='0':
                        top+=Coefficient_list[index]**2
                    elif qubit_term[target_qubit]=='1':
                        bottom+=Coefficient_list[index]**2
                    else:
                        print('MISTAKE')
                term_list.append({'control_state':CONTROL_state,'angle': np.arctan(np.sqrt(top/bottom))})
            alpha_j_dict[target_qubit]= term_list
    return alpha_j_dict

Get_control_parameters(2, [np.sqrt(0), np.sqrt(0), np.sqrt(0.1), np.sqrt(0.9)]) 

In [ ]:
target_qubit=2

n_control_qubits = target_qubit-1


state_list = [Get_state_as_str(n_control_qubits, i) for i in range(2**n_control_qubits)]
state_list

In [ ]:
def Get_state_prep_dict(num_qubits, Coefficient_list=None):

    if Coefficient_list is None:
        Coefficient_list= np.random.rand(2 ** num_qubits)

    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    alpha_j = {}
    for target_qubit_j in range(1,num_qubits+1):
        
        n_control_qubits = target_qubit_j-1
            
        CONTROL_state_list = [Get_state_as_str(n_control_qubits, i) for i in range(2**n_control_qubits)]
        
        state_toJneg1 = set([state[:qubit_j] for state in state_list])

        for term in state_toJneg1:
            upper_term = term + '1'
            lower_term = term + '0'

            upper_sum = []
            lower_sum = []

            for k in range(len(state_list)):
                state = state_list[k]
                if state[:qubit_j + 1] == upper_term:
                    upper_sum.append(Coefficient_list[k] ** 2)
                elif state[:qubit_j + 1] == lower_term:
                    lower_sum.append(Coefficient_list[k] ** 2)

            try:
                alpha_j[(qubit_j, term)] = np.arctan(np.sqrt(sum(upper_sum) / sum(lower_sum)))
            except:
                alpha_j[(qubit_j, term)] = 0

    return alpha_j

In [ ]:
def Get_state_prep_dict(num_qubits, Coefficient_list=None):

    if Coefficient_list is None:
        Coefficient_list= np.random.rand(2 ** num_qubits)

    state_list = [Get_state_as_str(num_qubits, i) for i in range(2 ** num_qubits)]
    alpha_j = {}
    for qubit_j in range(num_qubits):
        state_toJneg1 = set([state[:qubit_j] for state in state_list])

        for term in state_toJneg1:
            upper_term = term + '1'
            lower_term = term + '0'

            upper_sum = []
            lower_sum = []

            for k in range(len(state_list)):
                state = state_list[k]
                if state[:qubit_j + 1] == upper_term:
                    upper_sum.append(Coefficient_list[k] ** 2)
                elif state[:qubit_j + 1] == lower_term:
                    lower_sum.append(Coefficient_list[k] ** 2)

            try:
                alpha_j[(qubit_j, term)] = np.arctan(np.sqrt(sum(upper_sum) / sum(lower_sum)))
            except:
                alpha_j[(qubit_j, term)] = 0

    return alpha_j

In [ ]:

class State_Prep_Circuit(cirq.Gate):
    """
    Function to build cirq Circuit that will make an arbitrary state!

    e.g.:
   {
        (1, 0): 0.5092156980522868,
        (2, 0): 0.9097461710606383,
        (2, 1): 0.9338960671361634,
        (3, 0): 0.3007458481278772,
        (3, 1): 0.5945638342986989,
        (3, 2): 0.7174996992546281,
        (3, 3): 0.7105908988639925
    }

gives :

0: ── U = 0.51 rad ──(0)─────────────@──────────────(0)────────────(0)──────────────@────────────────@────────────────
                     │               │              │              │                │                │
1: ────────────────── U = 0.91 rad ── U = 0.93 rad ─(0)────────────@────────────────(0)──────────────@────────────────
                                                    │              │                │                │
2: ───────────────────────────────────────────────── U = 0.30 rad ─ U = 0.59 rad ─── U = 0.72 rad ─── U = 0.71 rad ───

    Args:
        circuit_param_dict (dict): A Dictionary of Tuples (qubit, control_val(int)) value is angle

    Returns
        A cirq circuit object to be used by cirq.Circuit.from_ops to generate arbitrary state

    """
    def __init__(self, circuit_param_dict):

        self.circuit_param_dict = circuit_param_dict

    def _decompose_(self, qubits):

        for Tuple in self.circuit_param_dict:

            theta = self.circuit_param_dict[Tuple]
            U_single_qubit = My_U_Gate(theta, 0)

            if Tuple[0]=='': #Tuple[0]==1:
                num_controls = 0
                control_values = []
            else:
                num_controls = Tuple[0] #Tuple[0] - 1
                control_values=[int(bit) for bit in Tuple[1]]

            # qubit_list = cirq.LineQubit.range(0,Tuple[0])
            qubit_list = qubits[0:Tuple[0]+1]
            yield U_single_qubit.controlled(num_controls=num_controls, control_values=control_values).on(*qubit_list)

    def _circuit_diagram_info_(self, args):

        max_qubit = max(Tuple[0] for Tuple in self.circuit_param_dict)
        string_list = []
        for i in range(max_qubit):
            string_list.append('state prep circuit')
        return string_list

    def num_qubits(self):
        max_qubit = max(Tuple[0]+1 for Tuple in self.circuit_param_dict) # +1 due to python indexing
        return max_qubit



if __name__ == '__main__':
    num_qub = 3
    # Coefficient_list = np.random.rand(2 ** num_qub) # [np.sqrt(1/(2**num_qub)) for _ in range(2**num_qub)]
    Coefficient_list = [np.sqrt(0.3), np.sqrt(0.1), np.sqrt(0.1), np.sqrt(0.1), np.sqrt(0.1), np.sqrt(0.1),
                        np.sqrt(0.1), np.sqrt(0.1)]  # [1/2,1/2,1/2,1/2]  #[0.9, 0.3, 0.3, 0.1]

    alpha_j = Get_state_prep_dict(num_qub, Coefficient_list=Coefficient_list)
    state_circ = State_Prep_Circuit(alpha_j)
    circuit = (cirq.Circuit(cirq.decompose_once((state_circ(*cirq.LineQubit.range(state_circ.num_qubits()))))))

    # circuit = cirq.Circuit(*circuit.all_operations(), *list(circuit.all_operations())[::-1])

    # MEASURE
    qubits_to_measure = (cirq.LineQubit(q_No) for q_No in range(num_qub))
    circuit.append(cirq.measure(*qubits_to_measure))
    print(circuit)

    # simulate
    simulator = cirq.Simulator()
    results = simulator.run(circuit, repetitions=100000)
    print(results.histogram(key='0,1,2')) # Need key to match number of qubits!!!
    print('actual state:')
    # NOTE! must not have any measurement (otherwise collapses state!)
    state_circ = State_Prep_Circuit(alpha_j)
    circuit = (cirq.Circuit(cirq.decompose_once((state_circ(*cirq.LineQubit.range(state_circ.num_qubits()))))))
    qubits_to_measure = (cirq.LineQubit(q_No) for q_No in range(num_qub))
    result = simulator.simulate(circuit, qubit_order=qubits_to_measure)
    print(np.around(result.final_state, 3))
    print('expected state amplitudes:', Coefficient_list)

    # alternative key method! key fined by ## HERE ###
    # # MEASURE
    # qubits_to_measure = (cirq.LineQubit(q_No) for q_No in range(num_qub))
    # circuit.append(cirq.measure(*qubits_to_measure, key='Z'))             ## HERE ### (can be any str)
    # print(circuit)
    #
    # # simulate
    # simulator = cirq.Simulator()
    # results = simulator.run(circuit, repetitions=1000)
    # print(results.histogram(key='Z'))                         ## re-use ###



In [ ]:
circuit = cirq.Circuit(cirq.X(cirq.LineQubit(1)),
                        cirq.X.controlled(num_controls=1,control_values=[0]).on(cirq.LineQubit(1),cirq.LineQubit(2)),
                       cirq.measure(cirq.LineQubit(1),cirq.LineQubit(2)))
print(circuit)
simulator = cirq.Simulator()
raw_result = simulator.run(circuit, repetitions=100)
hist_result = raw_result.histogram(key='1,2')
print(hist_result)

In [ ]:
def Get_wavefunction(quantum_circuit_no_M_gates, sig_figs=3):
    """
     Function to simulate quantum circuit and wavefunction

    """
    # quantum_circuit_M_gates_removed = quantum_circuit.moments[
    #                                   :-1]  # removes last moment (aka measurement step - which collapses wavefunction)
    # quantum_circuit_new = cirq.Circuit(quantum_circuit_M_gates_removed)
    simulator = cirq.Simulator()
    result = simulator.simulate(quantum_circuit_no_M_gates, qubit_order=quantum_circuit_no_M_gates.all_qubits())
    print(np.around(result.final_state, sig_figs))
    return result.final_state

In [ ]:
test2 = {1: [{'control_state': '0', 'angle': 0.9553166181245092},
  {'control_state': '1', 'angle': 0.8570719478501311}]}

state_circ_obj = State_Prep_Circuit(test2)
circuit = (cirq.Circuit(cirq.decompose_once((state_circ_obj(*cirq.LineQubit.range(state_circ_obj.num_qubits()))))))



full_circ = cirq.Circuit(cirq.X(cirq.LineQubit(0)), [*circuit.all_operations()])

print(full_circ)
from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('')
print('state')
dd= Get_wavefunction(full_circ)